In this notebook we test the variation of testing result after varying the input.

RoBERTa is used as the 6 way classifier.

First we take the Train Set and the Validation set

We randomise their order

we split them into 5 parts (folds) each having the same number of statements in each of the 6 classes.

Eventually 5 transformers will each be trained on 4 of the folds and 1 fold used for training

All 5 transformers will be trained on sets that differ gradually from each other

All will be tested on the same test set

And the result will be examined for the il-posed/ill-conditioned property as described in the thesis report

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import random
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

from simpletransformers.classification import ClassificationModel

In [2]:
# procedure for getting the data sets and formatting them for the transformer
 

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','RealTotal','Context'
],axis=1)
     
    return ReadSet

In [3]:
# preparing the training dataset
train=prepareDataset( 'train.xlsx')
# and display for inspecting
train

FileNotFoundError: [Errno 2] No such file or directory: 'train-clean.xlsx'

In [ ]:
# preparing the evaluation/validation dataset
Eval=prepareDataset('valid-clean.xlsx')
# and display for inspecting
Eval

To keep closer to the standard way of doing K-folds I am including the validation and training set together.

The data will be spilt in 5 folds.  At every trial 4 will be used for training and the other will be used for validation.

In [ ]:
train=train.append(Eval, ignore_index = True)
train

In [ ]:
# preparing the test set dataset
test=prepareDataset('test-clean.xlsx')
test

In [ ]:
# first we randomise the order of train

import random
 
randNum=[]

for row in range(len(train)):
    randNum.append(random.random())

train['RandNum']=randNum

train=train.sort_values(by=['RandNum'] )
train=train.drop(['RandNum'],axis=1)
del(randNum)
train
    

In [ ]:
count0=0
count1=0
count2=0
count3=0
count4=0
count5=0

train0=[] #all stamtements that are Pants on Fire  (class 0)
train1=[]
train2=[]
train3=[]
train4=[]
train5=[]


for row in range(len(train)):
        if train.iloc[row,1]==0: 
            count0+=1
            train0.append(train.iloc[row,:])
        if train.iloc[row,1]==1: 
            count1+=1
            train1.append(train.iloc[row,:])
        if train.iloc[row,1]==2: 
            count2+=1
            train2.append(train.iloc[row,:])
        if train.iloc[row,1]==3: 
            count3+=1
            train3.append(train.iloc[row,:])
        if train.iloc[row,1]==4: 
            count4+=1
            train4.append(train.iloc[row,:])
        if train.iloc[row,1]==5: 
            count5+=1
            train5.append(train.iloc[row,:])

print('0s ', count0)
print('1s ', count1)
print('2s ', count2)
print('3s ', count3)
print('4s ', count4)
print('5s ', count5)

            

In [ ]:
def div5(myinteger):
    size_m5 =myinteger-(myinteger%5)
    QuantityToRemove=size_m5/5
    
    return QuantityToRemove, myinteger%5

C0div5=div5(count0)
C1div5=div5(count1)
C2div5=div5(count2)
C3div5=div5(count3)
C4div5=div5(count4)
C5div5=div5(count5)



print('To omit from class 0s ', C0div5)
print('To omit from class 1s ', C1div5)
print('To omit from class 2s ', C2div5)
print('To omit from class 3s ', C3div5)
print('To omit from class 4s ', C4div5)
print('To omit from class 5s ', C5div5)

In [ ]:
train0

 The order is already for each set is already randomised.
We will now, for sets train0 to train5 split each into 5 roughly equal parts.

In [194]:
#train[class][fold]
train01=[]
train02=[]
train03=[]
train04=[]
train05=[] 
   

for row in range(len(train0)):
    
    if row<C0div5[0]: train01.append(train0[row])
        
    if row>=C0div5[0] and row<(C0div5[0]*2): train02.append(train0[row])
        
    if row>=(C0div5[0]*2) and row<(C0div5[0]*3): train03.append(train0[row])
    
    if row>=(C0div5[0]*3) and row<(C0div5[0]*4): train04.append(train0[row])
        
    if row>=(C0div5[0]*4) and row<(C0div5[0]*5): train05.append(train0[row])
        
    if row>=(C0div5[0]*5):
        train01.append(train0[row])
         
     

     



In [195]:
train01

[text      In April 1997, there was a gas out conducted n...
 labels                                                    0
 Name: 3359, dtype: object,
 text      The non-profit group Feeding America is lying ...
 labels                                                    0
 Name: 10012, dtype: object,
 text      My opponent has no child care plan.
 labels                                      0
 Name: 2996, dtype: object,
 text      Final election 2016 numbers: Trump won the pop...
 labels                                                    0
 Name: 1353, dtype: object,
 text      Pres. Obama's finance team is recommending a  ...
 labels                                                    0
 Name: 1725, dtype: object,
 text      I did not say that I would not have them Musli...
 labels                                                    0
 Name: 3959, dtype: object,
 text      Says Paul Ryan's budget plan ends Medicare.
 labels                                              0
 Name: 8431, dtyp

In [196]:
train02

[text      Says the Patient Protection and Affordable Car...
 labels                                                    0
 Name: 4893, dtype: object,
 text      Obama suggested bombing Pakistan.
 labels                                    0
 Name: 5088, dtype: object,
 text      We have my opponent Sam Olens to thank for the...
 labels                                                    0
 Name: 7653, dtype: object,
 text      I've been a cop in Lake County, Michigan, sinc...
 labels                                                    0
 Name: 9466, dtype: object,
 text      Obama would experiment with socialism.
 labels                                         0
 Name: 8749, dtype: object,
 text      I asked then-Secretary of State Condoleezza Ri...
 labels                                                    0
 Name: 909, dtype: object,
 text      Democrat Russ Feingold announced his Wisconsin...
 labels                                                    0
 Name: 7080, dtype: object,
 text

In [197]:
        
train11=[]
train12=[]
train13=[]
train14=[]
train15=[]
train1remaining=[]

train21=[]
train22=[]
train23=[]
train24=[]
train25=[]
train2remaining=[]

train31=[]
train32=[]
train33=[]
train34=[]
train35=[]
train3remaining=[]

train41=[]
train42=[]
train43=[]
train44=[]
train45=[]
train4remaining=[]

train51=[]
train52=[]
train53=[]
train54=[]
train55=[]
train5remaining=[]



In [198]:

for row in range(len(train1)):
    if row<C1div5[0]: train11.append(train1[row])
        
    if row>=C1div5[0] and row<(C1div5[0]*2): train12.append(train1[row])
        
    if row>=(C1div5[0]*2) and row<(C1div5[0]*3): train13.append(train1[row])
    
    if row>=(C1div5[0]*3) and row<(C1div5[0]*4): train14.append(train1[row])
        
    if row>=(C1div5[0]*4) and row<(C1div5[0]*5): train15.append(train1[row])
        
    if row>=(C1div5[0]*5):
        train11.append(train1[row])
         



In [199]:
for row in range(len(train2)):
    if row<C2div5[0]: train21.append(train2[row])
        
    if row>=C2div5[0] and row<(C2div5[0]*2): train22.append(train2[row])
        
    if row>=(C2div5[0]*2) and row<(C2div5[0]*3): train23.append(train2[row])
    
    if row>=(C2div5[0]*3) and row<(C2div5[0]*4): train24.append(train2[row])
        
    if row>=(C2div5[0]*4) and row<(C2div5[0]*5): train25.append(train2[row])
        
    if row>=(C2div5[0]*5):
        train21.append(train2[row])
         


In [200]:

for row in range(len(train3)):
    if row<C3div5[0]: train31.append(train3[row])
        
    if row>=C3div5[0] and row<(C3div5[0]*2): train32.append(train3[row])
        
    if row>=(C3div5[0]*2) and row<(C3div5[0]*3): train33.append(train3[row])
    
    if row>=(C3div5[0]*3) and row<(C3div5[0]*4): train34.append(train3[row])
        
    if row>=(C3div5[0]*4) and row<(C3div5[0]*5): train35.append(train3[row])
        
    if row>=(C3div5[0]*5):
        train31.append(train3[row])
         

In [201]:
for row in range(len(train4)):
    if row<C4div5[0]: train41.append(train4[row])
        
    if row>=C4div5[0] and row<(C4div5[0]*2): train42.append(train4[row])
        
    if row>=(C4div5[0]*2) and row<(C4div5[0]*3): train43.append(train4[row])
    
    if row>=(C4div5[0]*3) and row<(C4div5[0]*4): train44.append(train4[row])
        
    if row>=(C4div5[0]*4) and row<(C4div5[0]*5): train45.append(train4[row])
        
    if row>=(C4div5[0]*5):
        train41.append(train4[row])
         

In [202]:

for row in range(len(train5)):
    if row<C5div5[0]: train51.append(train5[row])
        
    if row>=C5div5[0] and row<(C5div5[0]*2): train52.append(train5[row])
        
    if row>=(C5div5[0]*2) and row<(C5div5[0]*3): train53.append(train5[row])
    
    if row>=(C5div5[0]*3) and row<(C5div5[0]*4): train54.append(train5[row])
        
    if row>=(C5div5[0]*4) and row<(C5div5[0]*5): train55.append(train5[row])
        
    if row>=(C5div5[0]*5):
        train51.append(train5[row])
         


In [203]:
pants1=pd.DataFrame(train01, columns=['text','labels'])
pants2=pd.DataFrame(train02, columns=['text','labels'])
pants3=pd.DataFrame(train03, columns=['text','labels'])
pants4=pd.DataFrame(train04, columns=['text','labels'])
pants5=pd.DataFrame(train05, columns=['text','labels'])


fake1=pd.DataFrame(train11, columns=['text','labels'])
fake2=pd.DataFrame(train12, columns=['text','labels'])
fake3=pd.DataFrame(train13, columns=['text','labels'])
fake4=pd.DataFrame(train14, columns=['text','labels'])
fake5=pd.DataFrame(train15, columns=['text','labels'])


Mfake1=pd.DataFrame(train21, columns=['text','labels'])
Mfake2=pd.DataFrame(train22, columns=['text','labels'])
Mfake3=pd.DataFrame(train23, columns=['text','labels'])
Mfake4=pd.DataFrame(train24, columns=['text','labels'])
Mfake5=pd.DataFrame(train25, columns=['text','labels'])


half1=pd.DataFrame(train31, columns=['text','labels'])
half2=pd.DataFrame(train32, columns=['text','labels'])
half3=pd.DataFrame(train33, columns=['text','labels'])
half4=pd.DataFrame(train34, columns=['text','labels'])
half5=pd.DataFrame(train35, columns=['text','labels'])


Mreal1=pd.DataFrame(train41, columns=['text','labels'])
Mreal2=pd.DataFrame(train42, columns=['text','labels'])
Mreal3=pd.DataFrame(train43, columns=['text','labels'])
Mreal4=pd.DataFrame(train44, columns=['text','labels'])
Mreal5=pd.DataFrame(train45, columns=['text','labels'])


real1=pd.DataFrame(train51, columns=['text','labels'])
real2=pd.DataFrame(train52, columns=['text','labels'])
real3=pd.DataFrame(train53, columns=['text','labels'])
real4=pd.DataFrame(train54, columns=['text','labels'])
real5=pd.DataFrame(train55, columns=['text','labels'])







frames1 = [pants2, pants3, pants4, pants5, fake2, fake3, fake4, fake5, Mfake2, Mfake3, Mfake4, Mfake5, half2, half3, half4, half5, Mreal2, Mreal3, Mreal4, Mreal5, real2, real3, real4, real5 ]

frames2 = [ pants1, pants3, pants4, pants5,fake1,  fake3, fake4, fake5,Mfake1, Mfake3, Mfake4, Mfake5,half1, half3, half4, half5,Mreal1, Mreal3, Mreal4, Mreal5,real1, real3, real4, real5 ]


frames3 = [pants1, pants2,  pants4, pants5,fake1, fake2, fake4, fake5,Mfake1, Mfake2, Mfake4, Mfake5,half1, half2,  half4, half5,Mreal1, Mreal2,  Mreal4, Mreal5,real1, real2,  real4, real5]


frames4 = [ pants1, pants2, pants3, pants5,fake1, fake2, fake3, fake5,Mfake1, Mfake2, Mfake3, Mfake5,half1, half2, half3, half5,Mreal1, Mreal2, Mreal3,  Mreal5,real1, real2, real3,  real5 ]

frames5 = [pants1, pants2, pants3, pants4, fake1, fake2, fake3, fake4,Mfake1, Mfake2, Mfake3, Mfake4, half1, half2, half3, half4,Mreal1, Mreal2, Mreal3, Mreal4, real1, real2, real3, real4]




train_fold1 = pd.concat(frames1)
train_fold2 = pd.concat(frames2)
train_fold3 = pd.concat(frames3)
train_fold4 = pd.concat(frames4)
train_fold5 = pd.concat(frames5)

#we set the omitted fold as the validation set

frames1=[pants1,fake1,Mfake1,half1,Mreal1,real1]
valid1 = pd.concat(frames1)

frames2=[pants2,fake2,Mfake2,half2,Mreal2,real2]
valid2 = pd.concat(frames2)

frames3=[pants3,fake3,Mfake3,half3,Mreal3,real3]
valid3 = pd.concat(frames3)

frames4=[pants4,fake4,Mfake4,half4,Mreal4,real4]
valid4 = pd.concat(frames4)

frames5=[pants5,fake5,Mfake5,half5,Mreal5,real5]
valid5 = pd.concat(frames5)
 

In [204]:
def randomiseSet(set):
    #this function randomises the order of the set.
    #order shouldn't be an issue but better keep things close to the realistic realms
 
    randNum=[]

    for row in range(len(set)):
        randNum.append(random.random())

    set['RandNum']=randNum
    set=set.sort_values(by=['RandNum'] )
    set=set.drop(['RandNum'],axis=1)
    
    return set

In [205]:
train_fold1=randomiseSet(train_fold1)
train_fold2=randomiseSet(train_fold2)
train_fold3=randomiseSet(train_fold3)
train_fold4=randomiseSet(train_fold4)
train_fold5=randomiseSet(train_fold5)
valid1=randomiseSet(valid1)
valid2=randomiseSet(valid2)
valid3=randomiseSet(valid3)
valid4=randomiseSet(valid4)
valid5=randomiseSet(valid5)

In [206]:
train_fold1.to_excel('./folds/train_fold1.xls')
train_fold2.to_excel('./folds/train_fold2.xls')
train_fold3.to_excel('./folds/train_fold3.xls')
train_fold4.to_excel('./folds/train_fold4.xls')
train_fold5.to_excel('./folds/train_fold5.xls')

valid1.to_excel('./folds/valid1.xls')
valid2.to_excel('./folds/valid2.xls')
valid3.to_excel('./folds/valid3.xls')
valid4.to_excel('./folds/valid4.xls')
valid5.to_excel('./folds/valid5.xls')

# We can now run the tests 

In [1]:
#Set the model being used here
model_class='roberta'  # bert or roberta or albert
model_version='roberta-large' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2
labels_count=6  # the number of classification classes



# Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import random
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

from simpletransformers.classification import ClassificationModel


#and function necessary for test set

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet




# preparing the test set dataset
test=prepareDataset('test.xlsx')

## Fold1 training and capturing predictions

In [2]:
fold_number='1'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval =pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [3]:

output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps, 
    "learning_rate": 1.2e-5,
    "train_batch_size": 32,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

# loading a previously saved ClassificationModel model based on this particular Transformer Class and model_name



model variables were set up: 


In [4]:
# loading the checkpoint that gave the best result
'''
CheckPoint='checkpoint-254-epoch-2'   


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )
''' 

"\nCheckPoint='checkpoint-254-epoch-2'   \n\n\npreSavedCheckpoint=output_folder+CheckPoint\n\nprint('Loading model, please wait...')\nmodel = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) \nprint('model in use is :', preSavedCheckpoint )\n"

In [5]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time taken: ", datetime.now() - current_time, ' at:',datetime.now())

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.776128

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.758651Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.784248Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.726642Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 1.748618

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.871441


Running loss: 1.576560Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 1.638866

Training of roberta model complete. Saved to ./folds/fold1/roberta/roberta-large/.
Training time taken:  0:21:39.910564  at: 2020-04-29 16:04:01.642268


In [6]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)



Converting to features started. Cache is not used.



{'mcc': 0.18332686383615832, 'acc': 0.33503032928942805, 'eval_loss': 1.5904662726863419}
Converting to features started. Cache is not used.



{'mcc': 0.11373107806799028, 'acc': 0.2800517018526497, 'eval_loss': 1.6736378988174543}


NameError: name 'test' is not defined

In [8]:
TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.11514695086910691, 'acc': 0.2829306313328137, 'eval_loss': 1.6509729785683713}
Test Set Result: 0.2829306313328137


In [9]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-2.2539062  -0.02435303 -0.09350586  1.1943359   1.1083984   0.60253906] 3   4 
[-0.00567627  0.39135742  0.16809082  0.0546875  -0.34155273 -0.6879883 ] 1   3 
[-0.45874023  0.49145508  0.15710449  0.02142334 -0.09143066 -0.05883789] 1   1 Match 1

[-1.1269531   0.39331055  0.44604492  0.8300781  -0.11425781 -0.2861328 ] 3   5 
[-1.9433594  -0.48168945 -0.1381836   0.7192383   1.34375     0.79052734] 4   5 
[-1.9091797   0.15246582 -0.5102539   0.8383789   0.9501953   0.9448242 ] 4   2 
[-2.         -0.41552734  0.09320068  0.93652344  1.3320312   0.67529297] 4   4 Match 2

[-0.95166016  0.3984375   0.31323242  0.57128906 -0.18469238  0.06445312] 3   5 
[-1.5400391e+00  2.1457672e-06 -4.0356445e-01  2.4096680e-01
  1.2070312e+00  1.2568359e+00] 5   4 
[-1.0751953   0.5595703   0.23254395  0.7890625   0.6015625   0.1472168 ] 3   5 
[-0.35131836  0.5078125   0.76953125  0.95947266 -0.12127686 -0.7495117 ] 3   5 
[-1.4863281   0.33544922  0.32055664  0.74902344  0.51708984  0.02937317] 

[-0.5878906   0.4638672   0.5253906   0.22387695 -0.36987305 -0.5126953 ] 2   1 
[-1.6289062   0.4555664   0.00226402  0.80371094  0.96972656  0.42651367] 4   1 
[-2.0546875  -0.04663086 -0.0949707   0.9814453   0.6435547   0.7553711 ] 3   4 
[-1.6015625   0.01110077  0.07470703  0.72753906  1.2578125   0.49682617] 4   5 
[-0.52001953  0.63378906  0.5019531   0.54785156  0.43359375 -0.5864258 ] 1   1 Match 40

[-1.6308594   0.09088135  0.19262695  0.99316406  0.46557617  0.21130371] 3   2 
[-1.0039062  -0.04925537 -0.29956055  0.5805664   0.24621582  0.88378906] 5   2 
[-0.27539062  0.31323242 -0.11657715  0.06469727 -0.29760742 -0.12609863] 1   1 Match 41

[-1.1914062   0.37817383  0.39233398  0.5029297   0.2680664  -0.01803589] 3   2 
[-1.6884766   0.01731873 -0.13000488  0.66845703  0.8808594   0.51953125] 4   1 
[-1.2597656   0.46679688  0.02558899  0.8120117  -0.0838623   0.27270508] 3   2 
[-1.3701172e+00 -1.0287762e-04 -1.9226074e-01  8.9404297e-01
  5.4296875e-01  9.1113281e-01

[-1.6513672   0.11846924 -0.16418457  0.7607422   0.9926758   0.45898438] 4   5 
[-1.9121094   0.03857422 -0.12756348  1.1503906   0.7832031   0.54003906] 3   4 
[-1.7558594  -0.06176758 -0.21777344  0.64746094  0.98339844  1.0205078 ] 5   4 
[-1.6943359  -0.04855347 -0.5180664   0.8613281   1.3310547   1.1611328 ] 4   1 
[-1.640625   -0.0960083  -0.03022766  1.1386719   0.76660156  0.23852539] 3   4 
[-0.4411621   0.17614746  0.49243164  0.60791016  0.13452148 -0.02749634] 3   3 Match 78

[ 0.02928162  0.6542969   0.7705078   0.42333984 -0.5961914  -1.1337891 ] 2   0 
[-1.3779297   0.49951172  0.39746094  0.86621094  0.29516602 -0.31201172] 3   3 Match 79

[-1.8242188  -0.3840332  -0.30688477  0.7885742   1.1640625   0.90771484] 4   3 
[-1.1552734   0.91259766  0.11340332  0.20605469  0.00316429 -0.06494141] 1   3 
[-1.8125     -0.37231445 -0.01791382  0.7661133   1.0556641   0.31079102] 4   4 Match 80

[-1.7431641   0.25341797  0.01441193  0.828125    0.7871094   0.34277344] 3   0 
[

[-1.5839844  -0.36865234 -0.3696289   1.0625      1.0253906   0.5107422 ] 3   4 
[-1.9248047  -0.42504883 -0.13317871  1.0361328   1.2382812   0.85595703] 4   3 
[ 0.08575439  0.5283203   0.13830566 -0.06799316 -0.40966797 -0.43969727] 1   5 
[-1.8759766   0.09844971  0.06256104  0.6386719   0.3239746   0.55078125] 3   1 
[-0.53466797  0.25       -0.06445312  0.10852051  0.07354736 -0.27734375] 1   2 
[-1.8662109  -0.01821899 -0.44091797  1.1064453   0.8300781   0.5048828 ] 3   4 
[-0.9819336   0.19238281  0.42626953  0.67578125  0.3564453  -0.16931152] 3   3 Match 134

[-2.1445312  -0.29858398 -0.37353516  1.0419922   1.3369141   0.8286133 ] 4   2 
[-1.4707031   0.2607422   0.44311523  0.95996094  0.21923828 -0.12017822] 3   4 
[-1.1533203   0.09002686  0.23962402  1.1650391   0.47973633 -0.3930664 ] 3   3 Match 135

[-0.24414062  0.09765625 -0.02874756 -0.02429199 -0.09240723 -0.11047363] 1   4 
[-1.1240234   0.32080078 -0.66259766  0.33764648  0.9399414   0.4375    ] 4   0 
[-2.1347

[ 0.81689453  0.7368164   0.43237305  0.01342773 -0.5751953  -1.3447266 ] 0   2 
[-0.4580078   0.5708008   0.6972656   0.2619629  -0.42993164 -0.51904297] 2   2 Match 178

[-0.859375    0.20214844  0.00361061  0.6928711   0.2286377  -0.12103271] 3   1 
[ 0.18908691  1.4033203   0.6621094   0.40893555 -0.37719727 -0.94091797] 1   0 
[-0.9511719   0.29248047  0.33862305  0.29223633  0.03219604 -0.05249023] 2   1 
[-1.8408203   0.27026367 -0.57177734  0.21350098  1.0283203   0.9633789 ] 4   1 
[-0.12322998  0.74902344  0.6010742   0.7753906  -0.46923828 -1.1240234 ] 3   2 
[-1.1972656   0.40234375 -0.18383789  0.48999023  0.48535156 -0.06768799] 3   5 
[-2.2871094  -0.11175537 -0.50878906  0.77246094  1.421875    0.66259766] 4   3 
[-2.0742188  -0.12585449 -0.35766602  0.6723633   1.5087891   0.7001953 ] 4   2 
[-1.4560547   0.33203125  0.58447266  1.1376953   0.7294922   0.00723648] 3   2 
[ 0.70703125  0.55859375  0.45385742  0.05786133 -0.43432617 -0.81689453] 0   0 Match 179

[-2.0625

[-0.1776123   0.79785156  0.7441406   0.41186523 -0.0559082  -0.6425781 ] 1   2 
[-0.9448242   0.39331055  0.22814941  0.09924316 -0.13330078 -0.25390625] 1   1 Match 219

[ 0.2861328   0.9082031  -0.09747314 -0.5463867  -0.74316406 -0.38842773] 1   4 
[-2.0664062  -0.06066895 -0.30932617  0.8286133   1.3974609   0.84765625] 4   4 Match 220

[-1.7236328   0.08081055 -0.05123901  0.53271484  0.7084961   0.6894531 ] 4   3 
[-2.0371094  -0.5410156  -0.43530273  1.0283203   1.6357422   1.0605469 ] 4   3 
[-0.54833984  0.48583984  0.18103027  0.19628906 -0.09655762 -0.07427979] 1   2 
[ 0.34545898  0.57421875 -0.32958984 -0.17468262 -0.3334961  -0.3852539 ] 1   0 
[-0.32739258  0.2919922  -0.14929199 -0.02792358 -0.24816895 -0.1496582 ] 1   1 Match 221

[-2.0429688  -0.11407471 -0.5493164   0.60546875  1.1972656   1.0800781 ] 4   4 Match 222

[-2.2246094  -0.18884277 -0.29516602  1.1611328   0.7866211   0.70458984] 3   4 
[-1.8486328   0.37280273 -0.15344238  0.5551758   1.2128906   0.69628

[-1.2353516   0.3305664   0.61865234  0.6113281   0.04202271 -0.34692383] 2   2 Match 261

[-1.5449219   0.07727051  0.21862793  0.37670898  0.703125    0.50683594] 4   2 
[-0.9995117   0.03546143  0.59472656  0.9716797   0.05023193 -0.45532227] 3   4 
[-0.23181152  0.6044922   0.11462402  0.02398682 -0.16516113 -0.6069336 ] 1   5 
[-0.49902344  0.3696289   0.22937012  0.2298584  -0.35766602 -0.2854004 ] 1   4 
[ 0.3935547   0.5473633   0.42700195 -0.21984863 -0.4272461  -0.75097656] 1   2 
[-0.86621094 -0.03591919  0.02096558  0.25390625  0.45336914  0.15466309] 4   4 Match 262

[-2.234375   -0.41137695 -0.07189941  1.1201172   1.1660156   0.72558594] 4   4 Match 263

[-1.3349609   0.14294434  0.34838867  1.0878906   0.5185547   0.00496292] 3   2 
[-0.19262695  0.82666016  0.03359985 -0.23205566 -0.46557617 -0.10778809] 1   3 
[-1.2851562   0.68066406  0.5839844   1.109375   -0.28076172 -0.54248047] 3   2 
[-1.4121094   0.15441895  0.0324707   0.27661133  0.1574707   0.5629883 ] 5   4


[-0.93359375  0.06738281 -0.27026367  0.22607422  0.09924316  0.4194336 ] 5   5 Match 302

[-1.4423828  -0.15563965  0.28320312  0.85791016  0.86328125  0.30737305] 4   1 
[-1.1875      0.46166992  0.546875    0.42626953  0.07873535  0.3557129 ] 2   4 
[-2.0820312  -0.34106445 -0.21582031  1.1738281   1.3291016   0.73046875] 4   2 
[-0.05181885  0.35424805 -0.046875   -0.14379883  0.15344238 -0.2364502 ] 1   5 
[-0.35302734  0.14990234  0.36401367  0.3076172  -0.00384521 -0.12817383] 2   1 
[-1.5166016   0.21179199 -0.4663086   0.1463623   0.8310547   1.0849609 ] 5   5 Match 303

[-1.7539062  -0.17272949 -0.1697998   0.8461914   1.1484375   0.65234375] 4   4 Match 304

[-0.05548096  0.5053711  -0.08984375 -0.18432617 -0.02914429 -0.05978394] 1   5 
[-0.08087158  0.29223633 -0.02073669 -0.12695312  0.09039307 -0.23742676] 1   4 
[-0.79785156  0.2890625   0.21057129  0.32104492 -0.1928711  -0.41577148] 3   1 
[-0.1940918   0.84375     1.1357422   0.84765625 -0.36889648 -0.99316406] 2   

[-0.47875977  0.11987305 -0.21374512 -0.00181293  0.26953125  0.00108528] 4   3 
[ 0.10791016  0.41015625  0.14050293 -0.2529297  -0.10168457 -0.49975586] 1   1 Match 342

[-0.6298828   0.61572266 -0.35131836  0.05316162 -0.00579453  0.36645508] 1   4 
[-0.23803711  0.59228516  0.44628906  0.10351562 -0.19897461 -0.5107422 ] 1   2 
[-2.0019531  -0.09783936 -0.48095703  0.93408203  1.6220703   0.7895508 ] 4   5 
[-0.82373047  0.2265625   0.15368652  0.27612305  0.24414062 -0.11444092] 3   5 
[-0.52490234  0.36450195  0.55810547  0.24194336 -0.5234375  -0.4807129 ] 2   2 Match 343

[-0.51904297  0.48828125  0.14404297  0.02902222 -0.12646484 -0.1315918 ] 1   2 
[-1.5888672  -0.13598633 -0.01600647  1.140625    0.90722656  0.12255859] 3   4 
[-1.5830078   0.2705078  -0.7558594   0.20751953  1.1552734   1.0800781 ] 4   4 Match 344

[-0.74121094  0.69384766  0.00966644  0.3232422   0.37158203  0.04388428] 1   1 Match 345

[ 0.12683105  0.48486328 -0.01343536 -0.10870361 -0.70410156 -0.28710

In [10]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  6  53   8  12  10   3]
 [  3 105  17  59  48  18]
 [  3  75  27  54  46   9]
 [  3  68  16  94  74  12]
 [  0  40  10  75 103  21]
 [  1  45   2  50  85  28]]


In [11]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.38      0.07      0.11        92
       False       0.27      0.42      0.33       250
 Barely-True       0.34      0.13      0.18       214
   Half-True       0.27      0.35      0.31       267
 Mostly-True       0.28      0.41      0.33       249
        True       0.31      0.13      0.19       211

    accuracy                           0.28      1283
   macro avg       0.31      0.25      0.24      1283
weighted avg       0.30      0.28      0.26      1283



In [12]:
Fold1_Predictions=pd.DataFrame(Pred, columns=['Pred1'])
Fold1_Predictions

,Pred1
0,3
1,1
2,1
3,3
4,4
...,...
1278,4
1279,1
1280,0
1281,1


In [13]:
Fold1_Predictions.to_excel(output_folder+'/Saves/fold1_Predictions.xls')

In [14]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 2: training & caturing predictions

In [3]:
fold_number='2'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [4]:

output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [6]:
# loading the checkpoint that gave the best result

CheckPoint='checkpoint-145-epoch-1'  #epoch 1


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )


Loading model, please wait...
model in use is : ./folds/fold2/roberta/roberta-large/checkpoint-143-epoch-1


In [5]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.798018

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.842521Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.773267

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.639407


Running loss: 1.647370Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.682445

Training of roberta model complete. Saved to ./folds/fold2/roberta/roberta-large/.
Training time:  0:11:03.365656


In [6]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.18907254261800885, 'acc': 0.338994050838291, 'eval_loss': 1.579885389771841}
Converting to features started. Cache is not used.



{'mcc': 0.10004978526635337, 'acc': 0.26776429809358754, 'eval_loss': 1.667520603640326}
Converting to features started. Cache is not used.



{'mcc': 0.12007635268138862, 'acc': 0.2852689010132502, 'eval_loss': 1.6483170883155163}
Training Result: 0.338994050838291
Eval Result: 0.26776429809358754
Test Set Result: 0.2852689010132502


In [7]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-2.8007812  -0.22363281 -0.3527832   1.3017578   1.171875    0.6875    ] 3   4 
[-0.8232422   0.35498047  0.4194336   0.4699707  -0.1842041  -0.44018555] 3   3 Match 1

[ 0.04122925  0.53125    -0.0317688   0.07269287 -0.37719727 -0.53759766] 1   1 Match 2

[-1.7041016  -0.16687012  0.16186523  1.2099609   0.4970703  -0.05978394] 3   5 
[-1.90625    -0.2548828  -0.36889648  1.0107422   0.89697266  0.45776367] 3   5 
[-1.4179688  -0.10241699 -0.17407227  0.7397461   0.4428711   0.14147949] 3   2 
[-1.2939453   0.07397461 -0.10900879  0.5024414   0.62109375  0.35009766] 4   4 Match 3

[-0.41259766  0.41333008  0.30737305  0.55566406 -0.10614014 -0.50097656] 3   5 
[-1.9726562  -0.10705566 -0.56591797  0.52197266  0.7338867   1.2041016 ] 5   4 
[-1.5605469   0.17663574 -0.04394531  0.8774414   0.54541016  0.08172607] 3   5 
[-0.9277344   0.35913086  0.0670166   0.9975586   0.10998535 -0.28930664] 3   5 
[-1.3251953   0.08239746  0.34545898  0.65185547  0.32617188 -0.24597168] 3   3 Match

[-1.6640625  -0.07141113  0.23657227  1.1835938   0.36669922 -0.1036377 ] 3   2 
[-1.2363281   0.4013672  -0.46728516 -0.02566528  0.30493164  0.52490234] 5   5 Match 34

[-1.8056641  -0.08917236  0.2932129   1.0996094   0.6660156   0.08215332] 3   4 
[-0.81396484  1.2480469  -0.08575439  0.27368164 -0.37597656 -0.8569336 ] 1   1 Match 35

[ 0.01035309  0.7680664   0.4572754   0.29882812 -0.45947266 -1.0058594 ] 1   0 
[-1.0146484   0.41796875 -0.26879883  0.13928223  0.07403564  0.2770996 ] 1   5 
[-1.6015625   0.07092285  0.48046875  1.0859375   0.6479492  -0.22021484] 3   1 
[ 0.49414062  0.84521484  0.7763672   0.95703125 -0.9238281  -1.4541016 ] 3   1 
[-1.5771484   0.01037598 -0.27001953  0.6933594   0.69140625  0.19885254] 3   3 Match 36

[-0.83691406  0.32666016 -0.16516113  0.25073242  0.05441284  0.1472168 ] 1   2 
[-0.02735901  0.63378906  0.54248047  0.2722168  -0.2644043  -0.94433594] 1   2 
[-2.6972656  -0.07879639 -0.78759766  0.8964844   1.1777344   0.85595703] 4   4 Ma

[-0.97216797  0.07678223 -0.46728516 -0.020401    0.53222656  0.45947266] 4   0 
[-2.3671875  -0.32788086 -0.15319824  1.2929688   0.9692383   0.49414062] 3   1 
[-0.7182617   0.1776123   0.17443848  0.2208252  -0.00154781  0.04562378] 3   3 Match 68

[ 0.29125977  0.5571289   0.7548828   0.02758789 -0.6298828  -1.03125   ] 2   5 
[ 0.6767578   0.83251953  0.09674072 -0.4272461  -0.46191406 -0.4230957 ] 1   0 
[-2.6425781   0.06317139 -0.77001953  0.9892578   1.2441406   1.2392578 ] 4   4 Match 69

[-0.61035156  0.3869629   1.0654297   0.8466797  -0.40625    -0.9975586 ] 2   2 Match 70

[ 0.28515625  0.6972656  -0.18847656 -0.11022949 -0.5761719  -0.6191406 ] 1   2 
[-0.3215332   0.5600586  -0.21875    -0.17150879 -0.03405762  0.30200195] 1   1 Match 71

[-0.61621094  0.25830078  0.7988281   0.7036133  -0.2939453  -0.6894531 ] 2   3 
[-0.39624023  0.55322266  0.68310547  0.5644531  -0.4880371  -0.5214844 ] 2   5 
[-0.9199219   0.3581543   0.6738281   0.49414062 -0.15686035 -0.6386719 ]

[-0.85546875  1.3134766  -0.30859375  0.52490234 -0.7207031  -0.46850586] 1   1 Match 105

[ 0.26245117  0.5996094   0.2932129  -0.30419922 -0.5439453  -0.34594727] 1   2 
[-1.4003906   0.07659912 -0.41333008  0.14208984  0.73583984  0.6777344 ] 4   0 
[ 1.0966797   0.7548828   0.44604492 -0.13232422 -1.2070312  -0.9765625 ] 0   4 
[-0.31762695  0.38549805  0.2512207   0.05334473 -0.4260254  -0.21972656] 1   1 Match 106

[-1.9892578  -0.04693604 -0.5253906   0.5073242   1.1337891   1.1601562 ] 5   4 
[-0.36206055  0.13891602 -0.01687622  0.17346191  0.07171631 -0.2524414 ] 3   1 
[-1.0361328   0.07781982 -0.13757324  0.21569824  0.49438477  0.36938477] 4   0 
[-0.44384766  0.48950195 -0.20043945 -0.17016602 -0.0848999   0.11877441] 1   1 Match 107

[-2.0351562  -0.2626953  -1.0810547   0.28857422  0.92333984  1.3359375 ] 5   5 Match 108

[-2.1113281  -0.12255859 -0.8769531   0.21984863  1.3349609   1.5332031 ] 5   4 
[-1.9238281  -0.10662842 -0.56884766  0.66015625  0.9482422   0.82177

[-2.0175781  -0.03582764 -0.50097656  1.0146484   0.90722656  0.59716797] 3   5 
[-1.1796875   0.49536133 -0.09283447  0.3359375   0.10327148 -0.08740234] 1   1 Match 145

[-0.96728516  0.13427734  0.43530273  0.5629883   0.15576172 -0.4609375 ] 3   2 
[-1.3193359   0.07910156 -0.05859375  0.5996094   0.4963379   0.2854004 ] 3   4 
[-1.8173828   0.0276947  -0.16967773  0.9008789   0.79345703  0.2919922 ] 3   2 
[ 0.50097656  0.69628906 -0.00895691 -0.55322266 -0.52685547 -0.06726074] 1   2 
[-0.29833984  0.6152344  -0.11151123 -0.14538574  0.03909302 -0.01161194] 1   3 
[-1.1513672   0.11199951 -0.5517578   0.15136719  0.41723633  0.47558594] 5   5 Match 146

[-0.7661133   0.6118164   0.28149414  0.37475586 -0.1003418  -0.4477539 ] 1   2 
[-1.7900391  -0.07489014 -0.29516602  0.7841797   0.94873047  0.51660156] 4   0 
[-2.7773438  -0.33251953 -0.5830078   1.3222656   1.2216797   0.90478516] 3   4 
[-1.1923828   0.6479492   0.5258789   0.68310547  0.30615234 -0.86865234] 3   1 
[-0.4523

...

[-2.5683594  -0.1459961  -0.5083008   0.83984375  1.5273438   0.73339844] 4   4 Match 216

[-1.5410156  -0.07305908 -0.14941406  0.2927246   0.7729492   0.4501953 ] 4   3 
[-2.7109375   0.03320312 -0.7734375   0.7241211   1.5742188   1.4443359 ] 4   3 
[-0.20739746  0.3359375   0.11450195  0.27734375 -0.06964111 -0.11700439] 1   2 
[ 0.4230957   0.55371094  0.09484863 -0.20910645 -0.5419922  -0.16992188] 1   0 
[-1.1445312   0.2775879  -0.01829529  0.41870117  0.32788086  0.32788086] 3   1 
[-2.0410156 -0.09375   -0.5073242  0.5517578  0.9008789  1.0507812] 5   4 
[-1.7929688  -0.23876953  0.35791016  1.1982422   0.5932617   0.06463623] 3   4 
[-1.8867188   0.14489746 -0.29711914  0.74609375  0.6645508   0.61621094] 3   4 
[-1.7607422  -0.1739502  -0.8339844   0.25048828  0.9682617   1.3984375 ] 5   5 Match 217

[-0.61816406  0.3696289   0.48535156  0.74121094 -0.18725586 -0.6582031 ] 3   3 Match 218

[-1.9492188  -0.13293457 -0.6035156   0.5654297   1.1660156   0.7285156 ] 4   2 
[-0.

[-2.6386719  -0.15368652 -0.5390625   0.82958984  1.3330078   0.77001953] 4   3 
[-1.9697266  -0.07574463 -0.19824219  0.49316406  1.0634766   0.8588867 ] 4   1 
[-1.0361328   0.24584961  0.06274414  0.40966797  0.28271484  0.08129883] 3   2 
[-1.3935547   0.08917236 -0.2265625   0.3059082   0.7050781   0.37353516] 4   3 
[-1.875      -0.05657959 -0.56152344  0.5834961   0.65625     0.91748047] 5   1 
[ 0.7685547   0.6274414   0.6171875  -0.14501953 -1.1152344  -1.1347656 ] 0   0 Match 254

[-1.7714844   0.03894043 -0.51708984  0.359375    0.63671875  1.0097656 ] 5   5 Match 255

[-2.3320312  -0.02636719 -0.44921875  0.9140625   1.3017578   0.7036133 ] 4   5 
[-1.5361328  -0.04736328 -0.6621094   0.21777344  0.6254883   0.6538086 ] 5   1 
[-0.46972656  0.42919922 -0.40966797 -0.12524414 -0.2019043   0.13745117] 1   1 Match 256

[-1.5898438   0.16699219  0.58447266  0.90966797  0.39819336 -0.4284668 ] 3   0 
[ 0.78271484  0.48217773  0.21447754 -0.3852539  -0.67089844 -0.47192383] 0   1

[-2.0351562   0.01018524 -0.39038086  0.92285156  0.7919922   0.43725586] 3   3 Match 288

[-1.3066406   0.12219238 -0.2788086   0.33544922  0.5834961   0.33935547] 4   3 
[ 0.3227539   0.60058594  0.47094727  0.02867126 -0.8911133  -0.76171875] 1   4 
[-2.1601562  -0.24072266 -0.14111328  0.8154297   1.1591797   0.6875    ] 4   1 
[-1.9609375   0.06420898 -0.60595703  0.4873047   0.7998047   1.0722656 ] 5   4 
[-2.2636719  -0.28271484 -0.73291016  0.62939453  1.4140625   0.96435547] 4   3 
[-1.65625     0.2565918  -0.63916016  0.09448242  0.6323242   1.1210938 ] 5   5 Match 289

[-1.1416016   1.2119141  -0.30395508  0.9970703  -0.5185547  -0.43237305] 1   1 Match 290

[-0.85791016  0.29418945 -0.08233643  0.47436523  0.07080078 -0.0980835 ] 3   1 
[-2.2753906  -0.10467529 -0.44702148  0.5859375   1.1455078   1.1044922 ] 4   5 
[-0.07672119  0.8129883   0.9770508   0.79589844 -0.6147461  -1.203125  ] 2   3 
[-1.4677734   0.23010254  0.01651001  0.62353516  0.4345703   0.1607666 ] 3   1

[-0.5756836   0.36865234  0.87060547  0.4946289  -0.16503906 -0.63378906] 2   2 Match 324

[-0.3630371   0.31958008  0.29492188  0.33081055 -0.34985352 -0.7709961 ] 3   1 
[-1.1513672   0.11950684 -0.2668457   0.38745117  0.17358398  0.5180664 ] 5   2 
[ 0.046875    0.44335938  0.14831543 -0.07330322 -0.23498535 -0.39453125] 1   1 Match 325

[-0.11315918  0.4165039   0.17749023  0.00250244 -0.20935059  0.01268005] 1   5 
[-0.92089844 -0.07415771  0.56103516  1.0078125  -0.05004883 -0.41674805] 3   2 
[ 0.02185059  0.4189453   0.2241211   0.12286377 -0.38378906 -0.54345703] 1   0 
[-1.6601562   0.13439941 -0.9560547  -0.0508728   0.5083008   1.4199219 ] 5   2 
[ 0.02636719  0.6899414  -0.01473999 -0.39135742 -0.29663086 -0.4309082 ] 1   5 
[-1.9628906  -0.11804199  0.10308838  0.78125     0.62158203  0.38623047] 3   3 Match 326

[-1.7382812   0.35888672 -0.65722656  0.33032227  0.7373047   1.1044922 ] 5   5 Match 327

[-0.5439453   0.05484009  0.19812012  0.07330322  0.16271973 -0.06488

[-0.45751953  0.4609375   0.01096344  0.33007812 -0.05770874 -0.4020996 ] 1   2 
[-0.64453125  0.33691406  0.8300781   0.6513672  -0.1138916  -0.7348633 ] 2   5 
[-0.16662598  0.21472168  0.27148438  0.14868164  0.02998352 -0.40527344] 2   5 
[-1.171875    0.01439667  0.19909668  0.48535156  0.28466797  0.04147339] 3   3 Match 356

[-1.1884766   0.24682617  0.27685547  0.53222656  0.22558594 -0.6088867 ] 3   3 Match 357

[ 0.3095703   0.828125    0.12194824 -0.296875   -0.29370117 -0.2475586 ] 1   2 
[-1.4707031  -0.04641724  0.05911255  0.39941406  0.5727539   0.25219727] 4   4 Match 358

[-1.3935547   0.03393555 -0.01290131  0.50390625  0.38330078 -0.0280304 ] 3   4 
[-0.05938721  0.48583984 -0.04043579  0.06341553 -0.18078613 -0.29003906] 1   3 
[-0.79785156  0.3310547   0.8569336   0.8305664  -0.27905273 -1.1201172 ] 2   1 
[-1.6025391   0.13635254  0.45751953  0.94921875  0.5288086  -0.27929688] 3   3 Match 359

[-1.0058594   0.15612793  0.5854492   0.4567871   0.31323242 -0.42480

In [8]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[ 17  28  15  13  13   6]
 [  3  77  27  78  45  20]
 [  4  54  34  71  34  17]
 [  6  50  20 105  64  22]
 [  1  27  11  87  88  35]
 [  4  31  11  46  74  45]]


In [9]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.49      0.18      0.27        92
       False       0.29      0.31      0.30       250
 Barely-True       0.29      0.16      0.20       214
   Half-True       0.26      0.39      0.31       267
 Mostly-True       0.28      0.35      0.31       249
        True       0.31      0.21      0.25       211

    accuracy                           0.29      1283
   macro avg       0.32      0.27      0.27      1283
weighted avg       0.30      0.29      0.28      1283



In [10]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred2'] )
Fold_Predictions

,Pred2
0,3
1,3
2,1
3,3
4,3
...,...
1278,4
1279,3
1280,0
1281,0


In [11]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold2_Predictions.xls')

In [12]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 3: training & caturing predictions

In [2]:
fold_number='3'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [5]:
 
output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 2,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [4]:
# loading the checkpoint that gave the best result
'''
CheckPoint='checkpoint-130-epoch-1'  #epoch 1


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )
'''

"\nCheckPoint='checkpoint-130-epoch-1'  #epoch 1\n\n\npreSavedCheckpoint=output_folder+CheckPoint\n\nprint('Loading model, please wait...')\nmodel = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) \nprint('model in use is :', preSavedCheckpoint )\n"

In [6]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.789267

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.756409Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.790398

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.799407Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.758912Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 1.779740Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 1.793229Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 1.735944Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Running loss: 1.811733


Running loss: 1.714102

Training of roberta model complete. Saved to ./folds/fold3/roberta/roberta-large/.
Training time:  0:11:03.825690


In [7]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.11407940929943007, 'acc': 0.2799351000540833, 'eval_loss': 1.6746232658109037}
Converting to features started. Cache is not used.



{'mcc': 0.0811927210366853, 'acc': 0.25433275563258234, 'eval_loss': 1.6956690525186473}
Converting to features started. Cache is not used.



{'mcc': 0.07621109676481715, 'acc': 0.2525331254871395, 'eval_loss': 1.6849112334074798}
Training Result: 0.2799351000540833
Eval Result: 0.25433275563258234
Test Set Result: 0.2525331254871395


In [8]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-2.1464844  -0.29663086 -0.3647461   0.57177734  0.8515625   0.47705078] 4   4 Match 1

[-0.15148926  0.28930664  0.1854248  -0.00510788 -0.27490234 -0.42651367] 1   3 
[-0.26757812  0.34570312  0.15686035  0.00333214 -0.04510498 -0.28735352] 1   1 Match 2

[-1.5449219  -0.08172607  0.21154785  0.41455078  0.19030762  0.26098633] 3   5 
[-2.1132812  -0.3737793  -0.25        0.82421875  0.6972656   0.6669922 ] 3   5 
[-1.2353516   0.07952881 -0.02888489  0.4658203   0.25756836  0.34399414] 3   2 
[-1.6289062  -0.21020508 -0.18139648  0.6245117   0.8129883   0.49243164] 4   4 Match 3

[-1.0234375   0.10650635  0.23376465  0.3017578   0.3515625   0.17810059] 4   5 
[-1.8300781  -0.22937012 -0.00797272  0.20324707  0.6801758   0.8144531 ] 5   4 
[-1.3759766  -0.0018568  -0.21398926  0.3623047   0.48339844  0.26049805] 4   5 
[-1.2958984   0.2052002  -0.13146973  0.56152344 -0.05459595 -0.38085938] 3   5 
[-1.265625    0.11572266  0.23706055  0.4807129   0.38842773  0.01602173] 3   3 Match

[-1.9345703  -0.32666016 -0.11907959  0.69189453  0.5839844   0.40356445] 3   4 
[-0.4560547   0.39697266  0.14770508  0.01448059 -0.08886719  0.06610107] 1   3 
[-0.65234375  0.38452148  0.2364502   0.11566162 -0.15270996 -0.00688553] 1   4 
[-1.5380859   0.0814209   0.39916992  1.0634766   0.08300781 -0.5107422 ] 3   4 
[-0.18017578  0.4169922   0.14758301 -0.10064697 -0.15124512 -0.09338379] 1   1 Match 55

[-1.1650391   0.12646484  0.27685547  0.47705078  0.14355469 -0.3232422 ] 3   4 
[-0.3708496   0.40429688  0.18139648  0.33935547 -0.26416016 -0.27685547] 1   2 
[-1.0615234   0.3046875   0.53027344  0.57910156  0.01282501 -0.62890625] 3   1 
[-1.9160156  -0.12286377  0.46313477  0.8125      0.625       0.19848633] 3   2 
[-0.35620117  0.30639648  0.09100342  0.03207397 -0.13439941 -0.10705566] 1   3 
[-1.1650391   0.25        0.15808105  0.35131836  0.45825195  0.02900696] 4   1 
[-1.9501953  -0.3527832  -0.11993408  0.7919922   0.70214844  0.0880127 ] 3   3 Match 56

[-0.189697


[-0.01983643  0.22961426  0.18688965  0.34960938 -0.28833008 -0.5239258 ] 3   1 
[-1.3867188  -0.10687256 -0.0802002   0.4958496   0.60009766  0.34765625] 4   2 
[-1.2265625  -0.12097168 -0.35717773  0.24902344  0.44555664  0.6430664 ] 5   4 
[-0.8544922   0.17199707  0.06329346  0.1607666   0.24291992  0.34155273] 5   1 
[-0.22314453  0.28344727  0.04626465  0.05102539 -0.2388916  -0.03231812] 1   1 Match 96

[-0.53271484  0.4338379  -0.02764893  0.00278282 -0.08746338 -0.06878662] 1   1 Match 97

[ 0.15551758  0.4428711   0.19995117 -0.12335205 -0.36010742 -0.40405273] 1   2 
[-0.58154297  0.28125     0.22839355  0.14501953  0.21569824  0.01960754] 1   0 
[ 0.07745361  0.38085938  0.1907959   0.05343628 -0.33911133 -0.4248047 ] 1   4 
[-1.0048828   0.46191406  0.0894165   0.5805664  -0.12963867 -0.59228516] 3   1 
[-1.8486328  -0.02400208 -0.18078613  0.64404297  0.9609375   0.7246094 ] 4   4 Match 98

[-0.5800781   0.21179199  0.16064453 -0.04397583 -0.1595459   0.19238281] 1   1 M

[-0.56103516  0.42700195  0.4501953   0.6508789  -0.08215332 -1.0732422 ] 3   2 
[-0.8671875   0.34545898  0.13134766  0.0218811   0.02668762  0.11004639] 1   4 
[-1.1230469   0.19873047  0.03256226  0.21252441  0.609375    0.08374023] 4   2 
[-0.1451416   0.32885742  0.09338379 -0.06216431 -0.17358398 -0.16491699] 1   2 
[-0.5756836   0.42993164  0.20153809  0.1842041  -0.0085907  -0.13366699] 1   3 
[-1.0126953   0.09185791 -0.14013672  0.15124512  0.28442383  0.33007812] 5   5 Match 132

[-0.3737793   0.45410156  0.18481445  0.19592285 -0.15759277 -0.43603516] 1   2 
[-1.6748047  -0.12683105 -0.11572266  0.5986328   0.6430664   0.56933594] 4   0 
[-2.2480469  -0.3330078  -0.39331055  0.8178711   0.8339844   0.75      ] 4   4 Match 133

[-1.1650391   0.17578125  0.47338867  0.58935547  0.20117188 -0.50341797] 3   1 
[-0.11694336  0.36523438  0.31323242  0.14111328 -0.10223389 -0.67626953] 1   1 Match 134

[ 0.63427734  0.70410156  0.11682129 -0.33691406 -0.58984375 -0.9404297 ] 1   0

[-0.3005371   0.22949219  0.19213867  0.15356445 -0.1685791  -0.03408813] 1   4 
[-0.7285156   0.29882812  0.30371094  0.36450195  0.04614258 -0.34692383] 3   2 
[-0.3630371   0.21569824  0.27441406  0.55078125 -0.4946289  -1.0146484 ] 3   4 
[-1.3818359  -0.1484375   0.51171875  1.2705078   0.04302979 -0.8930664 ] 3   2 
[-0.4873047   0.24938965  0.4309082   0.4741211  -0.00679398 -0.78466797] 3   2 
[-2.2773438  -0.2479248  -0.29858398  0.7597656   0.68896484  0.43603516] 3   2 
[-1.0986328   0.39013672  0.4946289   0.6230469   0.17382812 -0.4423828 ] 3   0 
[-0.09912109  0.40673828  0.55078125  0.28637695 -0.2524414  -0.8852539 ] 2   2 Match 171

[-0.46655273  0.28759766 -0.02085876 -0.14892578 -0.22705078 -0.2479248 ] 1   2 
[-1.4003906  -0.07995605  0.36938477  0.8911133   0.08288574 -0.0199585 ] 3   1 
[-1.3291016  -0.0814209  -0.0447998   0.26049805  0.4951172   0.61621094] 5   4 
[-1.0400391   0.05157471  0.10791016  0.03378296  0.28466797  0.4873047 ] 5   2 
[-0.9794922  -0.02

[-0.5683594   0.13598633  0.08599854  0.18395996 -0.20947266 -0.08111572] 3   2 
[-1.8154297  -0.3684082  -0.46777344  0.4724121   0.7866211   0.96777344] 5   1 
[-0.58984375  0.31982422  0.10968018  0.1920166  -0.02442932 -0.28051758] 1   5 
[-0.9536133   0.17883301  0.14428711  0.16882324  0.2253418   0.11450195] 4   4 Match 208

[-1.3574219  -0.05718994 -0.04125977  0.17785645  0.36987305  0.19421387] 4   5 
[-1.3300781   0.09240723  0.10968018  0.63916016  0.24145508  0.39819336] 3   3 Match 209

[-0.17089844  0.13293457  0.4482422   0.14050293 -0.24365234 -0.7260742 ] 2   1 
[-0.8676758   0.1071167   0.0869751  -0.00117111 -0.06689453  0.2565918 ] 5   2 
[-1.2890625  -0.17626953 -0.11322021  0.48266602  0.45507812  0.6845703 ] 5   2 
[-0.32885742  0.51220703  0.29296875  0.03665161 -0.27294922 -0.4765625 ] 1   3 
[-1.8427734  -0.12915039 -0.11352539  0.64501953  0.79541016  0.13623047] 4   5 
[-1.0664062   0.2841797   0.23803711  0.3239746   0.31176758 -0.24182129] 3   1 
[-0.4011

[-1.0019531   0.17919922 -0.24389648 -0.0534668   0.4572754   0.62841797] 5   5 Match 240

[-1.2324219  -0.06671143 -0.19836426  0.36523438  0.36279297  0.4399414 ] 5   0 
[-2.0683594  -0.20239258 -0.02502441  0.8701172   0.5908203   0.24060059] 3   5 
[-1.9140625  -0.17272949 -0.06915283  0.6142578   0.63378906  0.36132812] 4   3 
[-1.7148438  -0.12145996 -0.06311035  0.73583984  0.5253906   0.5834961 ] 3   4 
[ 0.0904541   0.34033203  0.02822876 -0.03044128 -0.15551758 -0.56689453] 1   3 
[-0.3918457   0.24475098  0.3305664   0.5205078  -0.22766113 -1.1132812 ] 3   3 Match 241

[-0.14575195  0.23925781  0.24353027  0.02032471 -0.3310547  -0.11169434] 2   4 
[-1.2324219   0.34521484  0.49414062  0.87158203  0.01535797 -0.8256836 ] 3   3 Match 242

[ 0.19921875  0.18432617  0.546875    0.17272949 -0.30541992 -0.95947266] 2   1 
[-0.64501953  0.19348145  0.01486206  0.15588379 -0.27001953 -0.05639648] 1   1 Match 243

[-0.8691406   0.08520508 -0.14941406 -0.14794922  0.14868164  0.38696

[-1.578125   -0.03085327 -0.16479492  0.52490234  0.7626953   0.40454102] 4   4 Match 278

[-0.8598633   0.33520508  0.421875    0.44384766  0.26586914 -0.34350586] 3   5 
[-1.8652344  -0.3010254  -0.48095703  0.47192383  0.9091797   0.88134766] 4   0 
[-1.7744141  -0.35986328 -0.12469482  0.2614746   0.62841797  0.60058594] 4   3 
[-1.6777344  -0.03421021 -0.1373291   0.38720703  0.4633789  -0.04238892] 4   5 
[-0.37109375  0.5625     -0.1817627  -0.04492188 -0.14257812  0.02371216] 1   2 
[ 0.2993164   0.26464844  0.34814453 -0.12298584 -0.42944336 -0.63720703] 2   3 
[ 0.08135986  0.35302734  0.08569336  0.0317688  -0.1928711  -0.57910156] 1   4 
[-1.4404297  -0.12512207 -0.21923828  0.36376953  0.3671875   0.7128906 ] 5   5 Match 279

[-0.7084961   0.03662109 -0.00613403  0.0058403   0.01583862  0.43701172] 5   3 
[-0.08221436  0.38134766  0.23205566 -0.02003479 -0.14892578 -0.33129883] 1   0 
[-0.5         0.32104492  0.26464844  0.4013672  -0.1463623  -0.7890625 ] 3   2 
[-0.1083

[-0.36938477  0.22351074  0.08746338 -0.03424072  0.01048279 -0.07116699] 1   0 
[-0.7246094  -0.01870728  0.18359375  0.04998779  0.20788574  0.17468262] 4   2 
[-0.4519043   0.54248047  0.2878418   0.17871094 -0.14050293 -0.18762207] 1   2 
[ 0.10247803  0.46313477  0.14282227 -0.15734863 -0.38061523 -0.47216797] 1   3 
[-0.2529297   0.2175293   0.04672241  0.20446777 -0.1887207  -0.5908203 ] 1   5 
[-0.9892578   0.20141602  0.03041077  0.5253906  -0.09155273  0.12634277] 3   4 
[-0.01364899 -0.02667236  0.18603516  0.23303223  0.05926514 -0.97265625] 3   2 
[-1.7509766  -0.04141235  0.0479126   0.77001953  0.24719238  0.0340271 ] 3   3 Match 320

[-0.6665039   0.38916016  0.16296387  0.25219727 -0.20751953 -0.02049255] 1   3 
[-0.5317383   0.43676758  0.23352051  0.32763672 -0.4116211  -1.1904297 ] 1   4 
[-2.0703125  -0.25805664 -0.07763672  0.58691406  0.9165039   0.62158203] 4   4 Match 321

[-0.39746094  0.05725098  0.3125      0.23803711 -0.00126171 -0.25585938] 2   2 Match 322

In [9]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  2  48  13  18   7   4]
 [  1  97  25  84  24  19]
 [  1  72  24  69  29  19]
 [  1  83  22 103  34  24]
 [  0  49  13  88  60  39]
 [  1  52  12  56  52  38]]


In [10]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.33      0.02      0.04        92
       False       0.24      0.39      0.30       250
 Barely-True       0.22      0.11      0.15       214
   Half-True       0.25      0.39      0.30       267
 Mostly-True       0.29      0.24      0.26       249
        True       0.27      0.18      0.21       211

    accuracy                           0.25      1283
   macro avg       0.27      0.22      0.21      1283
weighted avg       0.26      0.25      0.23      1283



In [11]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred3'] )
Fold_Predictions

,Pred3
0,4
1,1
2,1
3,3
4,3
...,...
1278,4
1279,3
1280,1
1281,1


In [12]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold3_Predictions.xls')

In [13]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 4: training & capturing predictions

In [4]:
fold_number='4'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [5]:

output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps, 
    "learning_rate": 1.8e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "adam_epsilon": 1e-8,
    "evaluate_during_training_steps": 5,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [6]:
# loading the checkpoint that gave the best result
 
CheckPoint='checkpoint-145-epoch-1' 


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )
 

Loading model, please wait...
model in use is : ./folds/fold4/roberta/roberta-large/checkpoint-145-epoch-1


In [7]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time, 'at: ' ,datetime.now())

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.690822

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.739897Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.742966Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.734375

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.621998Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 1.563075Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 1.613998

Training of roberta model complete. Saved to ./folds/fold4/roberta/roberta-large/.
Training time:  0:06:17.916872 at:  2020-04-30 14:22:59.287167


In [8]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./folds/fold4/roberta/roberta-large/cache/cached_dev_roberta_100_6_9245



{'mcc': 0.15667608410606707, 'acc': 0.3114115738236885, 'eval_loss': 1.6204105116504286}
Features loaded from cache at ./folds/fold4/roberta/roberta-large/cache/cached_dev_roberta_100_6_2308



{'mcc': 0.09219425181989767, 'acc': 0.25996533795493937, 'eval_loss': 1.70009144832348}
Features loaded from cache at ./folds/fold4/roberta/roberta-large/cache/cached_dev_roberta_100_6_1283



{'mcc': 0.11600529774543476, 'acc': 0.2805923616523772, 'eval_loss': 1.6730980505178004}
Training Result: 0.3114115738236885
Eval Result: 0.25996533795493937
Test Set Result: 0.2805923616523772


In [9]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-2.3125     -0.3305664  -0.11047363  1.140625    1.1982422   0.6586914 ] 4   4 Match 1

[-0.40527344  0.2849121   0.59472656  0.4580078  -0.35986328 -0.6743164 ] 2   3 
[-0.0074501   0.43676758  0.07275391 -0.15283203 -0.44702148 -0.22888184] 1   1 Match 2

[-1.1738281   0.05990601  0.42749023  0.70410156  0.19372559 -0.15490723] 3   5 
[-2.2910156  -0.45507812 -0.23779297  0.9223633   1.2705078   0.95410156] 4   5 
[-2.3027344  -0.3059082  -0.34204102  0.83154297  1.0966797   0.88134766] 4   2 
[-2.2539062  -0.3779297  -0.14404297  0.8569336   1.3271484   0.94140625] 4   4 Match 3

[-1.140625   -0.02842712  0.2619629   0.6010742   0.3881836  -0.13110352] 3   5 
[-1.9980469  -0.13476562 -0.46020508  0.35742188  0.9145508   1.0048828 ] 5   4 
[-1.4248047  -0.00334549 -0.06185913  0.6542969   0.5463867   0.09625244] 3   5 
[-1.015625    0.14978027  0.36132812  0.57958984  0.21484375 -0.38427734] 3   5 
[-1.8164062   0.06573486  0.08380127  0.83740234  0.61816406  0.09875488] 3   3 Match

[-0.04818726  0.4975586   0.4909668   0.22192383 -0.49169922 -0.5932617 ] 1   0 
[-0.87890625  0.50390625  0.0453186   0.2479248  -0.17248535  0.056427  ] 1   5 
[-1.6738281   0.0138855   0.24365234  0.5761719   0.51416016  0.11578369] 3   1 
[ 0.28881836  0.33520508  0.69189453  0.4387207  -0.6435547  -1.1064453 ] 2   1 
[-2.0234375  -0.24157715 -0.2770996   0.7680664   1.0498047   0.7524414 ] 4   3 
[-0.9873047   0.27807617  0.18969727  0.44970703 -0.00343704  0.05770874] 3   2 
[-0.37597656  0.52685547  0.71972656  0.55322266 -0.3479004  -0.9838867 ] 2   2 Match 34

[-2.2441406  -0.171875   -0.26757812  0.9848633   1.3691406   0.6660156 ] 4   4 Match 35

[-2.0253906  -0.17285156 -0.01365662  0.80859375  1.0693359   0.81103516] 4   4 Match 36

[-0.42578125  0.34716797  0.640625    0.6010742  -0.19067383 -0.7290039 ] 2   5 
[-1.4785156  -0.03170776  0.36376953  0.62158203  0.38964844 -0.14746094] 3   1 
[-1.3828125   0.32080078 -0.02792358  0.6376953   0.17797852  0.04473877] 3   1 
[

[-1.4013672   0.03274536  0.8076172   1.2792969   0.16674805 -0.59472656] 3   2 
[ 0.3161621   0.53808594 -0.19018555 -0.3071289  -0.5805664  -0.31591797] 1   2 
[-0.6796875   0.42358398 -0.00125122  0.0243988  -0.12841797  0.008255  ] 1   1 Match 66

[-1.2265625   0.03430176  0.66259766  1.1123047   0.38476562 -0.6245117 ] 3   3 Match 67

[-0.7055664   0.32128906  0.6611328   0.6928711  -0.24243164 -0.7006836 ] 3   5 
[-0.42407227  0.44604492  0.39257812  0.32055664 -0.2783203  -0.6503906 ] 1   4 
[-0.4477539   0.48339844 -0.0904541  -0.16052246 -0.23828125  0.09985352] 1   1 Match 68

[ 0.16955566  0.28344727  0.43164062  0.16772461 -0.60058594 -0.67041016] 2   0 
[-0.70654297  0.69921875 -0.1171875  -0.19104004 -0.30273438  0.12158203] 1   1 Match 69

[-1.4667969   0.23498535  0.85595703  1.4150391   0.24975586 -0.85839844] 3   3 Match 70

[-5.0634766e-01  3.3544922e-01 -5.1765442e-03 -4.7469139e-04
 -2.7001953e-01  1.4489746e-01] 1   2 
[-0.15222168  0.5854492   0.38598633  0.23962

[-0.78808594  0.13183594  0.5366211   0.5551758  -0.1472168  -0.58984375] 3   1 
[-2.3242188  -0.25952148 -0.30004883  0.58154297  1.4228516   1.4892578 ] 5   4 
[-0.26367188  0.34301758  0.0151825   0.07305908 -0.2614746   0.01997375] 1   1 Match 105

[-0.5698242   0.40283203 -0.14233398 -0.04278564  0.0209198   0.05355835] 1   0 
[-0.35961914  0.50146484 -0.16247559  0.00284386 -0.20947266  0.00178242] 1   1 Match 106

[-2.1035156  -0.03903198 -0.54345703  0.2788086   1.0917969   1.3642578 ] 5   5 Match 107

[-1.9775391  -0.11834717 -0.41992188  0.46020508  1.0107422   0.97314453] 4   4 Match 108

[-1.8847656  -0.3503418  -0.19702148  0.47265625  1.1044922   1.0947266 ] 4   5 
[-2.2382812  -0.43164062  0.23400879  1.1445312   0.9682617   0.23803711] 3   1 
[-2.4335938  -0.42773438 -0.31347656  1.0869141   1.4492188   1.0087891 ] 4   3 
[-2.2851562  -0.35473633 -0.27001953  0.7558594   1.4316406   1.2363281 ] 4   2 
[-1.7021484   0.08666992  0.1270752   0.64746094  0.63183594  0.07928

[-1.34375    -0.14233398 -0.34326172  0.11914062  0.5996094   0.9272461 ] 5   4 
[-0.13317871  0.3647461   0.6542969   0.43408203 -0.23449707 -0.82666016] 2   0 
[-2.40625    -0.1809082  -0.5498047   0.78271484  1.4677734   1.3935547 ] 4   4 Match 143

[ 0.95751953  0.55615234  0.00315094 -0.38989258 -1.0820312  -0.61083984] 0   3 
[-2.2773438  -0.3947754   0.33520508  1.2919922   1.0175781   0.13671875] 3   5 
[-0.51660156  0.27539062  0.4333496   0.49975586 -0.13208008 -0.4465332 ] 3   1 
[-0.6230469   0.31347656  0.90625     1.0136719  -0.22460938 -0.96875   ] 3   3 Match 144

[-2.0527344  -0.3701172  -0.53808594  0.52978516  1.1123047   1.1748047 ] 5   5 Match 145

[-1.0605469   0.20385742  0.01838684  0.05334473  0.17736816  0.33666992] 5   3 
[ 0.07403564  0.4711914   0.71777344  0.55615234 -0.6958008  -1.4355469 ] 2   1 
[-2.3476562  -0.40234375 -0.22570801  0.9013672   1.1933594   0.8300781 ] 4   4 Match 146

[-1.3427734   0.3828125  -0.21166992 -0.0736084   0.4255371   0.35009

...

[-1.1279297   0.1538086  -0.1529541   0.04696655  0.20031738  0.4729004 ] 5   3 
[-1.25       -0.05099487 -0.04916382  0.36279297  0.5151367   0.36816406] 4   2 
[-1.8662109   0.02197266 -0.5644531   0.08685303  0.8540039   1.1933594 ] 5   5 Match 218

[-1.2998047  -0.13793945 -0.22753906  0.15905762  0.6821289   0.7207031 ] 5   2 
[-0.24108887  0.3166504  -0.16601562 -0.38671875 -0.32763672  0.2524414 ] 1   1 Match 219

[-1.21875     0.41381836 -0.44604492  0.02926636  0.14880371  0.8173828 ] 5   3 
[-1.8847656  -0.01516724 -0.18908691  0.28442383  0.80615234  0.7470703 ] 4   3 
[-1.1005859   0.38964844 -0.45288086 -0.2956543   0.11358643  0.68359375] 5   1 
[-1.7294922   0.14990234  0.1932373   1.1855469   0.36279297 -0.1743164 ] 3   3 Match 220

[-2.1328125  -0.3791504  -0.23474121  0.8540039   1.2099609   0.84814453] 4   4 Match 221

[-0.64990234  0.40795898  0.6796875   0.66845703 -0.3466797  -0.5136719 ] 2   5 
[-2.1210938  -0.32641602 -0.10369873  0.6147461   1.1367188   0.92089

[-0.92529297  0.16174316  0.63623047  0.6645508   0.02055359 -0.46313477] 3   2 
[-1.0361328   0.32202148 -0.04620361  0.06555176  0.04693604  0.03222656] 1   2 
[-1.7226562  -0.03491211  0.65478516  1.1279297   0.44604492 -0.5703125 ] 3   4 
[-0.5083008   0.3996582   0.02827454  0.08459473  0.05307007  0.03549194] 1   5 
[-0.90283203  0.059021    0.11431885  0.45483398  0.18579102 -0.04129028] 3   4 
[ 0.34423828  0.4946289   0.25341797 -0.13171387 -0.6035156  -0.5722656 ] 1   2 
[ 3.0946732e-04  4.0185547e-01  3.3740234e-01  1.8176270e-01
 -3.5717773e-01 -3.5131836e-01] 1   4 
[-2.3691406  -0.40405273  0.13623047  1.3564453   1.2060547   0.39770508] 3   4 
[-1.4560547  -0.00216293  0.4963379   1.3212891   0.3864746  -0.6069336 ] 3   2 
[-0.21655273  0.33007812 -0.17224121 -0.18688965 -0.35131836  0.11804199] 1   3 
[-0.62841797  0.37426758  0.6689453   0.61083984 -0.31420898 -0.72021484] 2   2 Match 252

[-1.7666016  -0.26416016 -0.13049316  0.38989258  0.8491211   0.80078125] 4   4 

[-0.68115234  0.32543945  0.7319336   0.9628906  -0.1784668  -1.0117188 ] 3   1 
[-1.3457031   0.03286743  0.13842773  0.58496094  0.42211914  0.22424316] 3   3 Match 282

[-2.4335938  -0.3881836  -0.06774902  1.0390625   1.1513672   0.7080078 ] 4   4 Match 283

[-0.93652344  0.25390625  0.5214844   0.6591797   0.07189941 -0.54589844] 3   1 
[-1.5302734   0.05889893  0.8334961   1.3554688   0.25170898 -0.7216797 ] 3   3 Match 284

[-2.4863281  -0.27929688 -0.33447266  0.74365234  1.3662109   1.2382812 ] 4   5 
[-0.34472656  0.8178711   0.02862549 -0.12768555 -0.47436523 -0.3461914 ] 1   1 Match 285

[-2.1269531  -0.3083496  -0.32739258  0.63378906  1.2197266   1.1201172 ] 4   1 
[-2.0019531  -0.27441406  0.5234375   1.4140625   0.7314453  -0.4326172 ] 3   0 
[-1.7792969  -0.01708984 -0.42749023  0.3461914   0.8515625   1.0195312 ] 5   5 Match 286

[-2.1855469  -0.22167969 -0.6040039   0.64941406  1.3710938   1.296875  ] 4   3 
[-1.828125    0.01358032  0.04922485  0.8149414   0.6577148

[-0.8779297   0.06719971  0.07940674  0.25756836  0.2919922  -0.15515137] 4   4 Match 322

[ 0.22827148  0.1875      0.2644043  -0.00803375 -0.4272461  -0.5966797 ] 2   4 
[-0.6201172   0.4411621   0.9550781   1.0195312  -0.32421875 -1.3398438 ] 3   2 
[-2.0449219  -0.3708496   0.0836792   0.9902344   0.9916992   0.45458984] 4   2 
[ 0.21130371  0.44750977 -0.18261719 -0.3334961  -0.5058594   0.01348877] 1   2 
[-1.0009766   0.32592773  0.6611328   0.93066406  0.01617432 -0.7915039 ] 3   2 
[-1.0410156   0.3305664   0.15222168  0.42700195  0.07446289 -0.09307861] 3   3 Match 323

[-0.51464844  0.23706055 -0.11553955 -0.06677246 -0.09503174  0.19628906] 1   4 
[ 0.68408203  0.6953125   0.00491714 -0.12457275 -0.86816406 -0.6381836 ] 1   0 
[ 0.1574707   0.35424805  0.54541016  0.38623047 -0.609375   -0.7763672 ] 2   3 
[-1.4414062  -0.08996582 -0.2524414   0.22363281  0.5498047   0.6425781 ] 5   5 Match 324

[-1.7363281   0.07421875  0.28588867  0.94677734  0.3618164  -0.23571777] 3   3

[ 0.16296387  0.51123047 -0.19433594 -0.26049805 -0.44580078 -0.23925781] 1   1 Match 359

[-1.5292969  -0.01224518  0.8256836   1.2216797   0.33251953 -0.6401367 ] 3   2 
[ 0.27490234  0.44750977  0.001297   -0.27954102 -0.5654297   0.00128651] 1   0 
[-0.77734375  0.25634766 -0.18786621  0.01858521  0.28588867  0.1574707 ] 4   0 
[ 0.5102539   0.6713867   0.2631836  -0.12768555 -0.82470703 -0.9892578 ] 1   3 
[-1.4355469  -0.03582764  0.22058105  0.640625    0.45751953 -0.03561401] 3   2 
[-2.3964844  -0.3864746  -0.40771484  0.71875     1.3037109   1.0429688 ] 4   1 
[-0.6274414   0.17321777  0.37426758  0.29223633 -0.25       -0.19592285] 2   4 
[ 0.5029297   0.27172852  0.04620361 -0.21447754 -0.59277344 -0.47436523] 0   2 
[ 0.0869751   0.2607422   0.11199951 -0.21765137 -0.29370117 -0.15759277] 1   0 
[-0.09155273  0.3852539  -0.04885864 -0.15600586 -0.34814453 -0.10162354] 1   1 Match 360

360


In [10]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[ 17  33  10  15   9   8]
 [  6  73  21  86  39  25]
 [ 11  47  20  88  36  12]
 [ 10  44  18 103  63  29]
 [  5  27  10  69 100  38]
 [  5  30   7  49  73  47]]


In [11]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.31      0.18      0.23        92
       False       0.29      0.29      0.29       250
 Barely-True       0.23      0.09      0.13       214
   Half-True       0.25      0.39      0.30       267
 Mostly-True       0.31      0.40      0.35       249
        True       0.30      0.22      0.25       211

    accuracy                           0.28      1283
   macro avg       0.28      0.26      0.26      1283
weighted avg       0.28      0.28      0.27      1283



In [12]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred4'] )
Fold_Predictions

,Pred4
0,4
1,2
2,1
3,3
4,4
...,...
1278,4
1279,2
1280,0
1281,1


In [13]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold4_Predictions.xls')

In [14]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 5: training & capturing predictions

In [2]:
fold_number='5'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set

In [3]:
 
output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps,
    "learning_rate": 1e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "adam_epsilon": 1e-8,
    "evaluate_during_training_steps": 5,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

print(model_class,' model was set up')

roberta  model was set up


In [4]:
# loading the checkpoint that gave the best result

CheckPoint='checkpoint-145-epoch-1'   


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )


Loading model, please wait...
model in use is : ./folds/fold5/roberta/roberta-large/checkpoint-145-epoch-2


In [5]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.662294

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.700397Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.708131

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.662877Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.723657

Training of roberta model complete. Saved to ./folds/fold5/roberta/roberta-large/.
Training time:  0:05:39.796072


In [6]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./folds/fold5/roberta/roberta-large/cache/cached_dev_roberta_100_6_9245



{'mcc': 0.18191478143058618, 'acc': 0.33326122228231475, 'eval_loss': 1.5982493273527152}
Features loaded from cache at ./folds/fold5/roberta/roberta-large/cache/cached_dev_roberta_100_6_2308



{'mcc': 0.10592625541080404, 'acc': 0.27253032928942805, 'eval_loss': 1.6799660148291753}
Features loaded from cache at ./folds/fold5/roberta/roberta-large/cache/cached_dev_roberta_100_6_1283



{'mcc': 0.10298741616685525, 'acc': 0.2720187061574435, 'eval_loss': 1.658412375567872}
Training Result: 0.33326122228231475
Eval Result: 0.27253032928942805
Test Set Result: 0.2720187061574435


In [7]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-2.0136719  -0.22912598 -0.01447296  1.2871094   1.2089844   0.38256836] 3   4 
[-0.3203125   0.2446289   0.3623047   0.22070312 -0.35742188 -1.1679688 ] 2   3 
[-0.27978516  0.26342773 -0.03616333 -0.16748047 -0.30810547 -0.50439453] 1   1 Match 1

[-1.1503906  -0.07550049  0.5307617   0.9272461   0.12463379 -0.44628906] 3   5 
[-1.9873047  -0.5258789  -0.17297363  0.9848633   0.8149414   0.30810547] 3   5 
[-1.8955078  -0.13623047 -0.27856445  0.93310547  0.5991211   0.29418945] 3   2 
[-1.3496094  -0.28588867 -0.6821289   0.30078125  0.24633789  0.00396347] 3   4 
[-0.59716797  0.02363586 -0.33691406  0.07476807 -0.27734375 -0.46655273] 3   5 
[-1.8476562  -0.06280518 -0.45898438  0.29833984  1.0136719   1.5087891 ] 5   4 
[-1.3232422   0.15661621  0.31225586  0.87646484  0.50439453 -0.08294678] 3   5 
[-0.6142578  -0.05020142 -0.2626953   0.18713379 -0.3317871  -0.6088867 ] 3   5 
[-1.6767578   0.1829834   0.3461914   0.78808594  0.15063477 -0.15722656] 3   3 Match 2

[-0.41870117

[-0.26708984  0.27075195 -0.20568848 -0.17419434 -0.48168945 -0.13293457] 1   1 Match 45

[-1.0136719  -0.05007935 -0.00803375  0.78125     0.05773926 -0.3942871 ] 3   1 
[-1.2060547  -0.04714966 -0.04980469  0.08465576 -0.00685883  0.26000977] 5   0 
[ 0.0368042   0.13354492  0.49169922  0.2927246  -0.73876953 -0.9375    ] 2   0 
[-1.3613281  -0.10601807 -0.2758789  -0.01448059  0.10028076  0.6279297 ] 5   4 
[-0.73095703 -0.11431885 -0.3876953  -0.11157227 -0.20153809 -0.10876465] 5   1 
[-1.6416016   0.15527344 -0.36572266 -0.23522949  0.68603516  0.90722656] 5   5 Match 46

[-0.8305664   0.2397461   0.34472656  0.66259766 -0.18054199 -1.1494141 ] 3   2 
[-2.2109375  -0.12524414 -0.7138672   0.7573242   1.3291016   0.75      ] 4   5 
[-1.109375    0.5131836  -0.40576172  0.07897949  0.34936523  0.4326172 ] 1   5 
[-0.77734375 -0.15002441 -0.40429688  0.06964111 -0.06677246 -0.25927734] 3   0 
[-0.40478516  0.38354492  0.4091797   0.2626953  -0.39233398 -0.7788086 ] 2   1 
[-0.018173

[-1.296875   -0.08868408  0.38330078  0.6166992   0.20202637 -0.43676758] 3   3 Match 98

[-0.6459961   0.8017578   0.29296875 -0.06396484 -0.39746094 -0.7397461 ] 1   5 
[-0.92871094  0.05355835 -0.1352539   0.44506836  0.5444336   0.3005371 ] 4   4 Match 99

[ 0.2043457   0.5361328   0.23913574 -0.05044556 -0.7373047  -1.1953125 ] 1   1 Match 100

[-1.1494141   0.38745117  0.01931763  0.2076416   0.20581055 -0.1459961 ] 1   2 
[-1.9960938  -0.06970215 -0.20495605  0.26464844  0.90283203  1.1220703 ] 5   4 
[-1.5302734  -0.20471191 -0.11834717  0.6586914   0.39892578 -0.00901794] 3   1 
[-0.42260742  0.1105957   0.17211914  0.12341309 -0.50927734 -0.32495117] 2   1 
[-0.47338867  0.9506836  -0.4165039  -0.26489258 -0.39697266 -0.05636597] 1   1 Match 101

[ 0.28442383  0.55322266  0.18444824 -0.06427002 -0.88134766 -0.7890625 ] 1   2 
[-1.1630859   0.27661133  0.02392578 -0.00536346  0.06192017 -0.04498291] 1   0 
[ 0.41674805  0.4230957   0.06604004 -0.04174805 -0.9536133  -0.6083984

[-1.0507812   0.16882324  0.18579102  0.6713867   0.00746536 -0.7963867 ] 3   4 
[-0.29345703  0.21069336 -0.02490234  0.01184082 -0.2758789  -0.49975586] 1   3 
[-8.4277344e-01  8.6879730e-04  6.4794922e-01  1.3652344e+00
  3.3752441e-02 -1.6367188e+00] 3   3 Match 137

[-0.8208008  -0.20141602  0.22631836  0.75634766  0.05770874 -0.84375   ] 3   5 
[-1.59375    -0.07543945 -0.28564453  0.61621094  0.39208984  0.07580566] 3   4 
[-1.6894531  -0.17663574  0.0275116   0.97314453  0.8652344   0.2541504 ] 3   5 
[-1.0322266   0.40942383  0.28125     0.26416016  0.11755371 -0.15332031] 1   1 Match 138

[-0.15734863  0.33398438  0.45385742  0.16015625 -0.6611328  -1.2275391 ] 2   2 Match 139

[-1.1220703   0.06549072 -0.01408386  0.14038086  0.11846924 -0.04257202] 3   4 
[-1.4707031  -0.19262695  0.06915283  0.34570312  0.38012695  0.0894165 ] 4   2 
[ 0.4580078   0.4152832  -0.21398926 -0.30786133 -0.6586914  -0.49047852] 0   2 
[-1.1845703   0.2783203  -0.21508789 -0.2536621  -0.33642578

[-1.2060547   0.01027679  0.7128906   0.7836914   0.22399902 -0.12304688] 3   1 
[ 4.9471855e-06  5.3161621e-02  2.2705078e-01 -1.0888672e-01
 -4.5556641e-01 -8.6621094e-01] 2   3 
[-0.36938477  0.45214844  0.3647461   0.21289062 -0.2919922  -1.0185547 ] 1   3 
[-1.4179688   0.06176758 -0.12109375  0.77685547  0.82666016  0.35375977] 4   4 Match 175

[-0.8095703   0.03192139  0.36206055  0.3605957  -0.30688477 -1.0322266 ] 2   1 
[-1.3242188   0.04611206  0.6225586   0.74853516  0.05609131 -0.7338867 ] 3   5 
[-0.2602539   0.24133301  0.12145996  0.0635376  -0.26293945 -0.45483398] 1   3 
[-0.58691406  0.02764893 -0.06015015  0.10321045 -0.14343262 -0.45776367] 3   1 
[-0.86816406 -0.09490967 -0.01498413  0.3154297   0.17980957 -0.1352539 ] 3   4 
[-1.8876953  -0.18981934 -0.18640137  0.4091797   0.89453125  1.0146484 ] 5   5 Match 176

[-1.328125   -0.050354    0.19128418  0.4038086   0.49145508 -0.28100586] 4   5 
[-1.8330078  -0.08007812  0.34350586  0.85595703  0.5800781  -0.286621

[-1.8603516  -0.41015625 -0.06848145  1.0410156   1.1367188   0.73291016] 4   3 
[-0.84375    -0.11895752  0.05969238  0.32055664 -0.12683105 -0.0124588 ] 3   2 
[-0.29345703  0.26831055 -0.30151367 -0.3154297  -0.4387207  -0.34716797] 1   0 
[-0.63671875  0.08105469 -0.33422852  0.01477051 -0.24304199 -0.2388916 ] 1   1 Match 208

[-1.9394531   0.07830811 -0.54541016  0.59277344  1.1298828   1.3505859 ] 5   4 
[-1.9248047  -0.11669922  0.14367676  1.2353516   1.0507812   0.1694336 ] 3   4 
[-1.8310547  -0.11395264 -0.42114258  0.5366211   1.0742188   1.0371094 ] 4   4 Match 209

[-2.0898438  -0.20361328 -0.53466797  0.4338379   1.2275391   1.6640625 ] 5   5 Match 210

[-1.1845703   0.14477539  0.53759766  0.8857422  -0.04367065 -1.2832031 ] 3   3 Match 211

[-1.6689453  -0.12176514 -0.32373047  0.56884766  0.6386719   0.25512695] 4   2 
[-0.83935547  0.73339844 -0.62158203 -0.32983398  0.3869629   1.4414062 ] 5   5 Match 212

[ 0.3791504   0.5283203   0.12902832 -0.35668945 -0.8637695

[-1.9169922  -0.13134766 -0.27026367  0.89501953  0.94873047  0.7373047 ] 4   1 
[ 0.24609375  0.50097656  0.48120117  0.02183533 -0.93847656 -1.0498047 ] 1   0 
[-2.1308594   0.14428711 -0.43017578  0.05523682  0.5961914   1.1210938 ] 5   5 Match 243

[-1.9707031  -0.13330078 -0.04803467  1.2275391   1.1416016   0.46826172] 3   5 
[-2.046875   -0.28271484 -0.4140625   0.6069336   0.99853516  0.9482422 ] 4   1 
[-0.31103516  0.6635742  -0.38134766 -0.2841797  -0.33862305  0.2631836 ] 1   1 Match 244

[-1.4326172  -0.0100708   0.5180664   1.0332031  -0.04962158 -0.7158203 ] 3   0 
[ 0.01506042  0.2319336  -0.11688232 -0.28686523 -0.5336914  -0.5097656 ] 1   1 Match 245

[-1.1689453  -0.20532227  0.36157227  0.8198242  -0.02781677 -0.7607422 ] 3   3 Match 246

[ 0.6796875   0.66015625  0.18566895  0.01111603 -1.0058594  -1.1152344 ] 0   0 Match 247

[-1.5693359   0.17285156  0.8935547   1.1767578   0.41186523 -0.7324219 ] 3   4 
[-1.9560547  -0.08862305 -0.30004883  0.03543091  0.7749023

[-1.2070312   0.05709839  0.32080078  0.46801758 -0.26293945 -0.38745117] 3   3 Match 278

[-0.2006836   0.23168945  0.20751953  0.01229858 -0.37109375 -0.50390625] 1   0 
[-2.0917969  -0.25732422 -0.21411133  0.86621094  1.1738281   0.8457031 ] 4   1 
[-2.1191406  -0.62060547 -0.41015625  0.44433594  1.0195312   0.5029297 ] 4   5 
[-0.58203125  0.17419434  0.11083984  0.23876953 -0.3725586  -0.39916992] 3   2 
[-0.1315918   0.80908203  0.5385742   0.20043945 -0.5732422  -1.0263672 ] 1   3 
[-0.6298828   0.00748444  0.19787598  0.09173584 -0.38842773 -0.49389648] 2   4 
[-1.6708984   0.12573242  0.01603699  0.14196777  1.1230469   0.8671875 ] 4   2 
[-0.4885254   0.63183594  0.47558594  0.1361084  -0.5180664  -1.2246094 ] 1   1 Match 279

[-0.7949219  -0.27807617 -0.11206055  0.4699707   0.35083008 -0.7036133 ] 3   3 Match 280

[-1.9306641  -0.35180664  0.2524414   1.0654297   0.91796875  0.13757324] 3   4 
[-1.0361328   0.3137207   0.43676758  0.4038086  -0.32177734 -0.5751953 ] 2   1

[-1.0732422   0.04528809 -0.08343506  0.22265625  0.36865234 -0.21203613] 4   4 Match 315

[-1.8515625  -0.11529541 -0.50341797  0.2932129   1.1962891   1.6757812 ] 5   5 Match 316

[ 0.36083984  0.85791016 -0.35766602 -0.60498047 -0.8642578  -0.53515625] 1   5 
[ 0.24975586  0.11230469  0.16015625 -0.19018555 -0.6738281  -0.9345703 ] 0   5 
[-1.4189453   0.14904785 -0.21081543  0.01066589  0.62646484  0.5629883 ] 4   2 
[-2.2382812  -0.09423828 -0.26611328  0.8251953   1.1298828   1.0507812 ] 4   4 Match 317

[-1.3798828   0.2770996  -0.70751953  0.022995    0.72314453  1.359375  ] 5   2 
[ 0.19616699  0.42773438  0.09246826 -0.06240845 -0.16552734 -0.80859375] 1   4 
[-0.40161133  0.32299805  0.10772705  0.20214844 -0.17785645 -0.57910156] 1   4 
[-0.50634766 -0.04693604 -0.23950195  0.06347656 -0.3466797  -0.49072266] 3   2 
[-1.8896484  -0.2529297  -0.15856934  0.9760742   0.9355469   0.44799805] 3   2 
[ 0.14489746  0.11791992 -0.00170231 -0.11981201 -0.81396484 -0.6357422 ] 0   2

[-0.22937012  0.1817627   0.16101074  0.11706543 -0.2536621  -0.9580078 ] 1   5 
[-0.5058594   0.3857422   0.15258789  0.24621582 -0.16687012 -0.63427734] 1   4 
[-0.23413086  0.09997559  0.37036133  0.31640625 -0.32910156 -1.0976562 ] 2   2 Match 346

[-1.9150391  -0.36889648  0.07891846  0.5439453   0.8041992   0.2241211 ] 4   3 
[ 0.02394104  0.52783203 -0.04669189 -0.4819336  -0.6328125  -0.546875  ] 1   3 
[ 0.2241211   0.4321289   0.62402344  0.07635498 -0.5498047  -1.1015625 ] 2   4 
[-1.4423828  -0.15307617 -0.25341797  0.81591797  0.5732422  -0.12683105] 3   4 
[-0.73583984 -0.08013916  0.24108887  0.40283203 -0.15917969 -0.32983398] 3   2 
[-1.9785156  -0.1887207  -0.25585938  0.4729004   1.0117188   0.51708984] 4   3 
[-1.1376953   0.44091797 -0.06707764  0.58203125  0.21081543 -0.18164062] 3   2 
[ 0.18200684  0.34838867  0.41625977 -0.15039062 -0.70654297 -0.97216797] 2   5 
[ 0.25585938  0.60839844 -0.11584473 -0.46118164 -0.61328125 -0.7680664 ] 1   1 Match 347

[-1.5214

In [8]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  7  47  14  16   5   3]
 [  3 106  34  53  32  22]
 [  4  65  49  58  26  12]
 [  2  70  35  91  46  23]
 [  1  33  21  93  56  45]
 [  2  36  12  63  58  40]]


In [9]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.37      0.08      0.13        92
       False       0.30      0.42      0.35       250
 Barely-True       0.30      0.23      0.26       214
   Half-True       0.24      0.34      0.28       267
 Mostly-True       0.25      0.22      0.24       249
        True       0.28      0.19      0.22       211

    accuracy                           0.27      1283
   macro avg       0.29      0.25      0.25      1283
weighted avg       0.28      0.27      0.26      1283



In [10]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred5'] )
Fold_Predictions

,Pred5
0,3
1,2
2,1
3,3
4,3
...,...
1278,4
1279,1
1280,2
1281,1


In [11]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold5_Predictions.xls')

In [12]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

# Comparing the Predictions

In [15]:
Pred1=pd.read_excel('./folds/fold1/'+model_class+'/'+model_version+'/Saves/fold1_Predictions.xls')
Pred2=pd.read_excel('./folds/fold2/'+model_class+'/'+model_version+'/Saves/fold2_Predictions.xls')
Pred3=pd.read_excel('./folds/fold3/'+model_class+'/'+model_version+'/Saves/fold3_Predictions.xls')
Pred4=pd.read_excel('./folds/fold4/'+model_class+'/'+model_version+'/Saves/fold4_Predictions.xls')
Pred5=pd.read_excel('./folds/fold5/'+model_class+'/'+model_version+'/Saves/fold5_Predictions.xls')


In [16]:
 for row in range(len(Pred1)):
        
        print(Pred1.iloc[row,1] , end=',')
        print(Pred2.iloc[row,1]  , end=',')
        print(Pred3.iloc[row,1] , end=',')
        print(Pred4.iloc[row,1] , end=',')
        print(Pred5.iloc[row,1] )
    

3,3,4,4,3
1,3,1,2,2
1,1,1,1,1
3,3,3,3,3
4,3,3,4,3
4,3,3,4,3
4,4,4,4,3
3,3,4,3,3
5,5,5,5,5
3,3,4,3,3
3,3,3,3,3
3,3,3,3,3
1,1,1,1,1
1,1,1,0,1
4,5,5,5,5
2,3,1,2,1
5,4,5,4,3
5,3,1,3,2
1,3,3,3,3
4,3,4,4,4
3,3,3,4,3
4,4,3,4,4
1,5,5,5,1
3,2,2,3,2
1,1,1,3,3
3,1,1,1,1
3,1,3,1,3
1,4,4,4,2
3,3,2,3,3
3,3,1,3,3
3,3,1,3,1
4,5,5,4,4
3,0,1,0,0
3,3,3,3,3
3,5,5,3,3
5,3,2,3,3
1,1,5,5,5
3,1,1,3,3
5,3,1,3,3
3,4,4,1,5
1,1,1,1,2
3,3,1,3,1
3,3,5,5,4
3,3,1,3,3
5,1,1,5,1
1,3,1,3,3
5,4,5,5,4
2,2,2,3,2
3,3,4,3,3
5,3,1,5,3
5,4,5,4,3
4,3,5,4,3
3,3,3,3,3
4,1,1,1,1
5,5,1,5,1
3,3,1,3,3
5,5,5,4,5
3,2,3,1,2
4,4,5,5,1
1,1,1,3,1
1,1,1,3,1
4,4,4,3,4
1,1,1,1,1
3,1,2,3,3
1,1,1,0,3
4,3,1,2,3
3,4,4,4,3
5,5,5,5,5
1,2,2,2,1
4,3,1,1,1
3,5,1,4,3
1,3,1,1,2
3,3,3,3,2
1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
3,5,1,4,5
3,1,1,1,1
5,1,1,3,1
3,1,1,1,1
3,3,2,3,2
4,4,4,4,4
5,3,1,3,4
4,4,3,3,4
3,1,1,3,3
3,3,2,3,3
1,1,1,3,1
5,2,1,3,3
3,3,1,3,1
4,4,3,3,3
1,1,1,1,1
1,1,1,1,1
0,1,1,0,0
3,4,3,3,4
3,3,3,3,3
3,3,3,3,3
4,5,4,5,1
1,3,1,1,1
1,1,1,1,1
1,1,1,1,1


5,5,5,5,4
3,2,3,3,2
4,5,3,4,4
1,1,5,5,3
4,5,5,5,5
3,3,3,3,3
4,1,1,1,1
3,4,3,4,3
1,3,3,3,3
0,0,1,0,1
3,2,3,3,2
3,3,3,3,3
4,4,4,4,4
3,3,3,3,3
2,2,1,2,2
2,2,1,3,2
4,4,4,4,4
1,1,1,1,1
4,5,5,5,5
4,4,3,4,5
4,4,4,4,4
3,3,2,3,2
1,4,4,4,3
0,3,1,3,3
3,3,3,3,2
1,1,1,1,5
4,3,3,3,3
2,3,3,0,1
1,2,3,3,2
4,3,3,1,3
4,4,4,4,4
4,4,4,4,4
4,3,4,4,4
2,3,1,2,2
5,5,4,5,4
1,1,1,1,1
4,4,4,4,3
1,1,1,1,1
2,1,1,2,1
4,4,4,4,4
4,4,4,4,3
4,4,4,4,4
1,3,1,1,2
3,4,5,4,4
4,5,5,5,4
2,0,0,2,1
5,5,1,5,5
4,4,4,4,3
5,5,4,4,4
5,1,1,5,1
3,3,3,3,3
1,0,1,1,1
3,3,3,3,3
0,0,1,0,0
3,3,3,3,3
5,5,4,5,5
1,2,1,1,2
1,1,1,0,1
3,1,1,3,1
1,0,1,0,1
4,4,2,3,4
3,3,3,3,3
2,1,2,3,2
4,3,1,1,1
3,2,3,3,2
1,1,3,1,3
1,3,1,3,3
1,1,1,1,2
4,3,1,1,3
4,3,3,3,3
3,3,3,3,3
1,5,1,1,1
3,3,3,2,3
5,5,2,4,3
4,4,5,4,4
4,5,4,4,4
1,1,1,2,1
4,5,4,4,5
3,1,4,1,1
1,1,1,1,1
3,3,3,4,3
1,1,1,1,1
1,1,1,1,1
0,2,1,0,2
4,4,3,4,4
3,3,3,4,3
1,1,1,3,2
1,1,2,1,1
5,5,1,5,5
3,3,1,3,2
4,5,5,4,4
3,3,3,3,4
4,4,3,4,3
1,3,1,1,1
1,1,1,1,1
1,1,1,1,1
4,3,3,4,3
1,1,2,1,1
4,4,3,4,5
4,4,5,4,4


In [17]:
results=pd.DataFrame( columns=['text', 'label','fold1','fold2','fold3','fold4','fold5'])

results['text']=test['text']
results['label']=test['labels']
results['fold1']=Pred1['Pred1'] 
results['fold2']=Pred2['Pred2'] 
results['fold3']=Pred3['Pred3'] 
results['fold4']=Pred4['Pred4'] 
results['fold5']=Pred5['Pred5'] 

        
results

,text,label,fold1,fold2,fold3,fold4,fold5
0,New Mexico was 46th in teacher pay (when he wa...,4,3,3,4,4,3
1,Barack Obama and Hillary Clinton have changed ...,3,1,3,1,2,2
2,I'll tell you what I can tell this country: If...,1,1,1,1,1,1
3,Tommy Thompson created the first school choice...,5,3,3,3,3,3
4,Fifty-six percent decline in overall crime. A ...,5,4,3,3,4,3
...,...,...,...,...,...,...,...
1278,"We have trade agreements with 20 countries, an...",1,4,4,4,4,4
1279,On Donald Trumps plan to cut federal funding t...,4,1,3,3,2,1
1280,"Black Lives Matter, who are attacking law enfo...",2,0,0,1,0,2
1281,Latina who enthusiastically supported Donald T...,0,1,0,1,1,1


In [18]:
now=datetime.now()

results.to_excel('./folds/'+model_class+'_results'+now.strftime("%m-%d-%Y %H-%M")+'.xls' ,index=False)